# Model Selection, Functions, Loops, and parallel processing

While it is true that you want to get a "feel" of what is happening during the analysis, sometimes you just wish you could run a chunk of code and walk away. Loops are great for that and can even be run in parallel. Here's an example of how you can use them to help you make a decision.

## Typical way beginners would look into model selection

\# 1. DBH

test=lmer(dbh~(1|type), data = tree_df)


AIC(test)

\# AIC is 958.1

test=lmer(dbh~classe_drainage+(1|type), data = tree_df)

AIC(test)

\# AIC is 802.7

test=lmer(dbh~D_index+(1|type), data = tree_df)

AIC(test)

\# AIC is 867.3

test=lmer(dbh~defoliation+(1|type), data = tree_df)

AIC(test)

\# AIC is 960.1

test=lmer(dbh~epid_age+(1|type), data = tree_df)

AIC(test)

\# AIC is 958.1

## What if your variables change? What if you want to store your results?
### Loops and functions can help you!

In [ ]:
# load packages
packgs<-c("rJava", "glmulti", "lme4", "data.table")
lapply(packgs, library, character.only=TRUE)

In [ ]:
files <- grep(list.files(path = "../data/yearly_files/",pattern = ""), pattern='function_', inv=T, value=T)

In [ ]:
files = paste("../data/yearly_files/", files, sep="")

In [ ]:
temp <- lapply(files, fread, sep=",")

In [ ]:
surveys <- rbindlist( temp )

In [ ]:
plots=read.csv("../data/plots_details.csv", header=T)

In [ ]:
species=read.csv("", header=T)

In [ ]:
m1=merge(surveys, species, by="")

In [ ]:
survey_df=merge(m1, plots, by="")

In [ ]:
# wrap function around lmer
lmer.wrap<-function(formula,data,random="",...){
  lmer(paste(deparse(formula),random),data=data, REML=FALSE,...)
}

In [ ]:
# glmulti is used here for model selection
par_function <- function(y) {
  name<-paste(y)
  out <- tryCatch({
    glmulti(formula(paste(y, "dist_from_river+dbh+height+surface_terriere+pc_conif", sep="~")), 
            data=survey_df, random="+(1|)", fitfunc=lmer.wrap, intercept=TRUE, confsetsize = 10, level=1)
  }, error = function(error) {
    print(paste("ERROR:  ", error))
    return(NA)
  })
  return(out@objects[[1]])
}

In [ ]:
par_function("")

### An even better option

In [ ]:
#list of variables
yvar.list<-list("hindfoot_length", "weight") # 3 response variables
xvar<-"dist_from_river+dbh+height+surface_terriere+pc_conif"# several explanatory variables which will be combined later
output.list=list()

In [ ]:
t.st<-Sys.time()
for(y in yvar.list){
  out<-glmulti(formula(paste(y, "dist_from_river+dbh+height+surface_terriere+pc_conif", sep="~")), 
                data=survey_df, random="+(1|)", fitfunc=lmer.wrap, intercept=TRUE, confsetsize = 10, level=1)
  output.list[[y]]<-out@objects[[1]]
}
t.fin<-Sys.time()
print(t.fin-t.st)

### We can make it even better by using all available resources

In [ ]:
# cluster creation
#library(Rmpi)
library(snow)
library(doSNOW)
library(foreach)

In [ ]:
# Single node
library(doParallel)
cl<-makeCluster(8)
registerDoParallel(cl)

In [ ]:
# Export variables and libraries
out <- clusterEvalQ(cl, library(rJava))
out <- clusterEvalQ(cl, library(glmulti))
out <- clusterEvalQ(cl, library(lme4))
out <- clusterExport(cl, 'lmer.wrap')
out <- clusterExport(cl, 'glmulti')
out <- clusterExport(cl, 'survey_df')
out <- clusterExport(cl, 'yvar.list')
out <- clusterExport(cl, 'xvar')

In [ ]:
foreach.loop<-foreach(y=yvar.list, .combine="cbind") %dopar% par_function(y)
saveRDS(foreach.loop, 'output_foreach.rds')

In [ ]:
#####################
# delete cluster
#####################
stopCluster(cl)

In [ ]:
con <- gzfile("output_foreach.rds")
readRDS(con)
close(con)

# If you are running your script on our servers


In [ ]:
nCore = as.integer(Sys.getenv("MOAB_PROCCOUNT"))
if (is.na(nCore)){ nCore = 8 }

# Multiple nodes
cl = makeMPIcluster(nCore)
registerDoSNOW(cl)